In [1]:
import pathlib

import numpy as np
import pandas as pd

In [2]:
# Parameters
cell_type = "SHSY5Y"
aggregation = True
nomic = True

In [ ]:
if aggregation == True & nomic == True:
    aggregated_data_path = pathlib.Path(
        f"../data/{cell_type}_preprocessed_sc_norm_aggregated_nomic.parquet"
    )
elif aggregation == False & nomic == True:
    aggregated_data_path = pathlib.Path(
        f"../data/{cell_type}_preprocessed_sc_norm_nomic.parquet"
    )
elif aggregation == True & nomic == False:
    aggregated_data_path = pathlib.Path(
        f"../data/{cell_type}_preprocessed_sc_norm_aggregated.parquet"
    )
elif aggregation == False & nomic == False:
    pass
else:
    raise ValueError("Wrong parameters")

In [10]:
path = pathlib.Path(f"../data/{cell_type}_preprocessed_sc_norm.parquet")

data_df = pd.read_parquet(path)

data_df.head()

if nomic == True:
    # import nomic data
    nomic_df_path = pathlib.Path(
        f"../2.Nomic_nELISA_Analysis/Data/clean/Plate2/nELISA_plate_430420_{cell_type}_clean.csv"
    )
    df_nomic = pd.read_csv(nomic_df_path)

    # drop columns that contain [pgML]
    df_nomic = df_nomic.drop(
        columns=[col for col in df_nomic.columns if "[pgML]" in col]
    )
    # drop first 25 columns
    df_nomic = df_nomic.drop(columns=df_nomic.columns[3:25])
    df_nomic = df_nomic.drop(columns=df_nomic.columns[0:2])
else:
    df_nomic = None

In [11]:
df_nomic

,position_x,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],...,VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU],Treatment,Dose,oneb_Treatment_Dose_Inhibitor_Dose,twob_Treatment_Dose_Inhibitor_Dose,threeb_Treatment_Dose_Inhibitor_Dose,fourb_Treatment_Dose_Inhibitor_Dose
0,C13,0.595518,0.564662,0.538727,0.751911,0.796710,0.506570,0.557007,0.000000,0.130110,...,0.570920,0.344809,0.232809,0.725177,DMSO,0.10_%,DMSO_0.100_%_DMSO_1.000_%,DMSO_DMSO_0.10_%,DMSO__0.10_%__DMSO_1.0,DMSO__0.10_%__DMSO__1.0
1,D13,0.549004,0.856272,0.127459,0.070602,0.810357,0.359850,0.600601,0.682277,0.288149,...,0.457524,0.225962,0.300728,0.459638,DMSO,0.10_%,DMSO_0.100_%_Z-VAD-FMK_100.000_uM,DMSO_Z-VAD-FMK_0.10_%,DMSO__0.10_%__Z-VAD-FMK_100.0,DMSO__0.10_%__Z-VAD-FMK__100.0
2,E13,0.891845,0.640559,0.620414,0.142671,0.187645,0.000000,0.440517,0.651019,0.548641,...,0.685494,0.207687,0.754146,0.679904,DMSO,0.10_%,DMSO_0.100_%_Z-VAD-FMK_30.0_uM,DMSO_Z-VAD-FMK_0.10_%,DMSO__0.10_%__Z-VAD-FMK_30.0,DMSO__0.10_%__Z-VAD-FMK__30.0
3,F13,0.739737,0.987796,0.123841,0.579838,0.483562,0.599216,0.454416,0.407962,0.568372,...,0.537461,0.483328,0.533404,0.187470,DMSO,0.10_%,DMSO_0.100_%_Z-VAD-FMK_100.000_uM,DMSO_Z-VAD-FMK_0.10_%,DMSO__0.10_%__Z-VAD-FMK_100.0,DMSO__0.10_%__Z-VAD-FMK__100.0
4,G13,0.489414,0.684927,0.359885,0.529060,0.684803,0.484723,0.714058,0.532762,0.488618,...,0.338754,0.748028,0.447766,0.733610,DMSO,0.10_%,DMSO_0.100_%_Z-VAD-FMK_30.0_uM,DMSO_Z-VAD-FMK_0.10_%,DMSO__0.10_%__Z-VAD-FMK_30.0,DMSO__0.10_%__Z-VAD-FMK__30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,M15,0.537553,0.454080,0.121235,0.537197,0.299965,0.656053,0.495266,0.404481,0.483184,...,0.000000,0.754374,0.676954,0.570847,Topotecan,5_nM,Topotecan_5.000_nM_DMSO_0.025_%,Topotecan_DMSO_5_nM,Topotecan__5_nM__DMSO_0.03,Topotecan__5_nM__DMSO__0.03
150,B13,0.972764,0.738218,0.401194,0.722364,0.043239,0.412779,0.457284,0.683110,0.335358,...,0.385698,0.176153,0.727225,0.529847,media_ctr,0_nan,media_ctr_0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_0_nan,media_ctr__0_nan__Media_ctr_0.0,media_ctr__0_nan__Media_ctr__0.0
151,I13,0.725803,0.560143,0.383278,0.320750,0.668227,0.520634,0.669621,0.314319,0.460774,...,0.518688,0.428738,0.442567,0.486011,media_ctr,0_nan,media_ctr_0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_0_nan,media_ctr__0_nan__Media_ctr_0.0,media_ctr__0_nan__Media_ctr__0.0
152,O22,0.356083,0.349350,0.335341,0.790661,0.250488,0.487088,0.237995,0.878783,0.226819,...,0.509006,0.663276,0.597001,0.403857,media_ctr,0_nan,media_ctr_0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_0_nan,media_ctr__0_nan__Media_ctr_0.0,media_ctr__0_nan__Media_ctr__0.0


In [12]:
# subset each column that contains metadata
metadata = data_df.filter(regex="Metadata")

# get all columns that are not metadata except for metadata_Well
data = data_df.drop(metadata.columns, axis=1)

# get the metadata_Well column
metadata_well = metadata[
    ["Metadata_Well", "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
]

data_df = pd.merge(data, metadata_well, left_index=True, right_index=True)

In [21]:
df_nomic.drop(
    columns=[
        "Treatment",
        "Dose",
        "twob_Treatment_Dose_Inhibitor_Dose",
        "threeb_Treatment_Dose_Inhibitor_Dose",
        "fourb_Treatment_Dose_Inhibitor_Dose",
    ],
    inplace=True,
)

In [24]:
if (aggregation == True) and (nomic == True):

    # subset each column that contains metadata
    metadata = data_df.filter(regex="Metadata")
    data_df = data_df.drop(metadata.columns, axis=1)
    data_df = pd.concat([data_df, metadata["Metadata_Well"]], axis=1)
    # groupby well and take mean of each well
    data_df = data_df.groupby("Metadata_Well").mean()
    # drop duplicate rows in the metadata_well column
    metadata = metadata.drop_duplicates(subset=["Metadata_Well"])
    # get the metadata for each well
    data_df = pd.merge(
        data_df, metadata, left_on="Metadata_Well", right_on="Metadata_Well"
    )
    data_df = pd.merge(
        data_df,
        df_nomic,
        left_on=["Metadata_Well", "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"],
        right_on=["position_x", "oneb_Treatment_Dose_Inhibitor_Dose"],
    )
    data_df = data_df.drop(columns=["position_x"])
    # drop all metadata columns
    labeled_data = data_df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    data_x = data_df.drop(metadata.columns, axis=1)
    # set path to save the data
    aggregated_data_path = pathlib.Path(
        f"../data/{cell_type}_preprocessed_sc_norm_aggregated_nomic.parquet"
    )


elif (aggregation == True) and (nomic == False):
    # subset each column that contains metadata
    metadata = data.filter(regex="Metadata")
    data_df = data_df.drop(metadata.columns, axis=1)
    data_df = pd.concat([data_df, metadata["Metadata_Well"]], axis=1)
    # groupby well and take mean of each well
    data_df = data_df.groupby("Metadata_Well").mean()
    # drop duplicate rows in the metadata_well column
    metadata = metadata.drop_duplicates(subset=["Metadata_Well"])
    # get the metadata for each well
    data_df = pd.merge(
        data_df,
        df_nomic,
        left_on=["Metadata_Well", "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"],
        right_on=["position_x", "oneb_Treatment_Dose_Inhibitor_Dose"],
    )
    # set path to save the data
    aggregated_data_path = pathlib.Path(
        f"../data/{cell_type}_preprocessed_sc_norm_aggregated.parquet"
    )
elif (aggregation == False) and (nomic == True):
    data_df = pd.merge(
        data_df,
        df_nomic,
        left_on=["Metadata_Well", "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"],
        right_on=["position_x", "oneb_Treatment_Dose_Inhibitor_Dose"],
    )
    data_df = data_df.drop(columns=["position_x"])
    # set path to save the data
    aggregated_data_path = pathlib.Path(
        f"../data/{cell_type}_preprocessed_sc_norm_with_nomic.parquet"
    )
elif aggregation == False and nomic == False:
    pass
else:
    print("Error")

/tmp/ipykernel_174922/2378442375.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_df = data_df.groupby("Metadata_Well").mean()


In [ ]:
# set path to save the data

# save the data
data_df.to_parquet(aggregated_data_path)